In [1]:
# Plan:

# Class: TikZPicture
# Members:
# - points (coordinates)
# - objects

# Methods:
# - produce TeX code
# - add point (dot, no dot)
# - add line (normal, thick, dashed)
# - add label

In [176]:
import uuid

class TikZPicture:
    
    def __init__(self):
        self.coordinates = {}
        self.objects = []
        
    # Add named coordinates to picture
    def addCoordinate(self, x, y, name=None):
        if name == None:
            n = "c" + str(uuid.uuid1().int)[-5:]
        else:
            n = name
            
        self.coordinates[n] = (x,y)
        return n
        
    def getCoordinate(self, coordinate):
        if type(coordinate) == str:
            cname = coordinate
        else:
            cname = self.addCoordinate(coordinate[0], coordinate[1])
        return cname
        
    # Add a point with an optional label
    def addPoint(self, coordinate, label="", nodeOptions=[]):

        cname = self.getCoordinate(coordinate)
        
        p = {"kind":"point",\
             "label":label,\
             "coordinate":cname,\
             "nodeOptions":nodeOptions}
        self.objects.append(p)
        
    # Add a label with no point
    def addLabel(self, coordinate, label, options=[]):
            
        cname = self.getCoordinate(coordinate)
            
        p = {"kind":"label",\
             "label":label,\
             "coordinate":cname,\
             "options":options}
        self.objects.append(p)
        
        
    # Add a line with at least two points
    def addLine(self, coords, options=[], angles=[]):
        
        # For each coordinate, construct it if no name
        cs = []
        for c in coords:
            cs.append(self.getCoordinate(c))
        
        p = {"kind":"line",\
             "coords":cs,\
             "options":options,\
             "angles":angles}
        self.objects.append(p)
        
    # Draw Axes (0,y) -> (0,0) -> (x,0)
    def addAx(self, x, y, xlabel="", ylabel="", options=[]):
        
        self.addCoordinate(0, y, "yax")
        self.addCoordinate(x, 0, "xax")
        
        p = {"kind":"axes",\
            "x":"xax",\
            "y":"yax",\
            "xlabel":xlabel,\
            "ylabel":ylabel}
        self.objects.append(p)
        
    def addPlot(self, formula, domain, options=[]):
        
        p = {"kind":"plot",\
             "formula":formula,\
             "domain":domain,\
             "options":options}
        self.objects.append(p)
        
    # Output TikZ Code
    # do all this at end, so we build the TikZ snippet in
    # the right order because TeX is dumb
    def build(self):
        tikzString = ""
        tikzString += "\\begin{tikzpicture}\n"
        
        # Add coordinates
        for c in self.coordinates.keys():
            tikzString += "\\coordinate (%s) at (%f,%f);" \
                % (c, self.coordinates[c][0], self.coordinates[c][1]) + "\n"
        
        # Add objects
        for o in self.objects:
            
            if o['kind'] == "axes":
                tikzString += "\\draw[<->] (%s) node[above] {%s} -- (0,0) -- (%s) node[right]{%s}" \
                    % (o['y'], o['ylabel'], o['x'], o['xlabel']) + ";\n"
                    
            if o['kind'] == "point":
                tikzString += "\\filldraw [black] (%s) circle (2pt) node[" % (o['coordinate'])
                
                # Add node options
                for i, n in enumerate(o['nodeOptions']):
                    if i != 0:
                        tikzString += ", "
                    tikzString += n
                
                tikzString += "] {%s}" \
                    % (o['label']) + ";\n"
                    
            if o['kind'] == "label":
                tikzString += "\\node["
                
                for opt in o['options']:
                    tikzString += opt + ", "
                
                tikzString += "] at (%s) {%s}" \
                    % (o['coordinate'], o['label']) + ";\n"
                    
            if o['kind'] == "line":
                tikzString += "\\draw ["
                
                # Add options one by one
                
                for opt in o['options']:
                    tikzString += opt + ", "
                
                tikzString += "]  "
                
                # Add coordinates one by one
                
                # Add angles if smooth plot
                if 'smooth' in o['options']:
                    for i, c in enumerate(o['coords']):
                        if i != 0:
                            tikzString += " to [out=%f,in=%f] " % o['angles'][i-1]
                        tikzString += "(%s) " % (c)
                    
                # Add straight lines if not curved
                else:
                    for i, c in enumerate(o['coords']):
                        if i != 0:
                            tikzString += " -- "
                        tikzString += "(%s) " % (c)
                
                tikzString += ";\n"
                
            if o['kind'] == "plot":
                tikzString += "\\draw ["
                
                # Add options one by one
                
                for opt in o['options']:
                    tikzString += opt + ", "
                
                # Add domain, variable, and formula
                
                tikzString += "domain=%f:%f] plot (\\x, {%s});\n" % \
                    (o['domain'][0], o['domain'][1], o['formula'])
        
        tikzString += "\\end{tikzpicture}\n"
        
        return tikzString

In [141]:
[0,1] + [2]

[0, 1, 2]

In [129]:
t = TikZPicture()
t.addAx(11, 11, xlabel="Quantity", ylabel="Price")

t.addLine([(2,2), (10,10)])
t.addLine([(2,10), (10,2)])
t.addLine([(0,6), (6,6)], options=['dashed'])
t.addLine([(6,0), (6,6)], options=['dashed'])

t.addLabel((11,10), "Supply")
t.addLabel((11,2), "Demand")

t.addPoint((0,6), "$p^*$", nodeOptions=['left'])
t.addPoint((6,0), "$q^*$", nodeOptions=['below'])

print(t.build())

\begin{tikzpicture}
\coordinate (c61139) at (11,2);
\coordinate (c36627) at (2,2);
\coordinate (c42515) at (2,10);
\coordinate (c35315) at (6,0);
\coordinate (yax) at (0,11);
\coordinate (xax) at (11,0);
\coordinate (c50995) at (0,6);
\coordinate (c73683) at (6,6);
\coordinate (c54419) at (0,6);
\coordinate (c71379) at (6,0);
\coordinate (c49619) at (10,2);
\coordinate (c64435) at (11,10);
\coordinate (c94355) at (10,10);
\coordinate (c14547) at (6,6);
\draw[<->] (yax) node[above] {Price} -- (0,0) -- (xax) node[right]{Quantity};
\draw []  (c36627)  -- (c94355) ;
\draw []  (c42515)  -- (c49619) ;
\draw [dashed ]  (c50995)  -- (c14547) ;
\draw [dashed ]  (c71379)  -- (c73683) ;
\node[draw] at (c64435) {Supply};
\node[draw] at (c61139) {Demand};
\filldraw [black] (c54419) circle (2pt) node[left] {$p^*$};
\filldraw [black] (c35315) circle (2pt) node[below] {$q^*$};
\end{tikzpicture}



In [139]:
t = TikZPicture()
t.addAx(11, 11, xlabel="Peas", ylabel="Carrots")

t.addLine([(0,8), (8,0)], options=['dashed'])
t.addLine([(2,10), (4,4), (10,2)], options=['smooth'], angles=[(-90,135), (-45,180)])

t.addPoint((0,8), "$\\frac{Y}{p_c}$", nodeOptions=['left'])
t.addPoint((8,0), "$\\frac{Y}{p_p}$", nodeOptions=['below'])

t.addPoint((4,4), "$(q_p^*,q_c^*)$", nodeOptions=['above right'])

print(t.build())

\begin{tikzpicture}
\coordinate (c50867) at (2,10);
\coordinate (yax) at (0,11);
\coordinate (c94547) at (10,2);
\coordinate (xax) at (11,0);
\coordinate (c55795) at (0,8);
\coordinate (c08435) at (0,8);
\coordinate (c94579) at (8,0);
\coordinate (c51955) at (4,4);
\coordinate (c31891) at (4,4);
\coordinate (c91763) at (8,0);
\draw[<->] (yax) node[above] {Carrots} -- (0,0) -- (xax) node[right]{Peas};
\draw [dashed ]  (c55795)  -- (c91763) ;
\draw [smooth ]  (c50867)  to [out=-90,in=135] (c51955)  to [out=-45,in=180] (c94547) ;
\filldraw [black] (c08435) circle (2pt) node[left] {$\frac{Y}{p_c}$};
\filldraw [black] (c94579) circle (2pt) node[below] {$\frac{Y}{p_p}$};
\filldraw [black] (c31891) circle (2pt) node[above right] {$(q_p^*,q_c^*)$};
\end{tikzpicture}



In [178]:
t = TikZPicture()
t.addAx(11, 11, xlabel="$k$", ylabel="")

t.addPlot("4.5*ln(\\x+1)", [0,10]) # income
t.addPlot("3*ln(\\x+1)", [0,10])   # investment
t.addLine([(0,0), (10,10)])        # depreciation

t.addLine([(5.711,0), (5.711,10)], options=['dashed'])
t.addLabel((5.711,0), "$k^*$", options=['below'])

t.addLabel((10,10), "$(n+g+\delta)k$", options=['right'])
t.addLabel((10,10.8), "$f(k)$", options=['right'])
t.addLabel((10,7.12), "$sf(k)$", options=['right'])

print(t.build())

\begin{tikzpicture}
\coordinate (c75315) at (0.000000,0.000000);
\coordinate (c65779) at (10.000000,10.800000);
\coordinate (c40435) at (5.711000,0.000000);
\coordinate (c56147) at (5.711000,10.000000);
\coordinate (xax) at (11.000000,0.000000);
\coordinate (c12499) at (10.000000,7.120000);
\coordinate (c09011) at (10.000000,10.000000);
\coordinate (c03347) at (10.000000,10.000000);
\coordinate (c86675) at (5.711000,0.000000);
\coordinate (yax) at (0.000000,11.000000);
\draw[<->] (yax) node[above] {} -- (0,0) -- (xax) node[right]{$k$};
\draw [domain=0.000000:10.000000] plot (\x, {4.5*ln(\x+1)});
\draw [domain=0.000000:10.000000] plot (\x, {3*ln(\x+1)});
\draw []  (c75315)  -- (c09011) ;
\draw [dashed, ]  (c40435)  -- (c56147) ;
\node[below, ] at (c86675) {$k^*$};
\node[right, ] at (c03347) {$(n+g+\delta)k$};
\node[right, ] at (c65779) {$f(k)$};
\node[right, ] at (c12499) {$sf(k)$};
\end{tikzpicture}



In [20]:
x = uuid.uuid1()
str(x)

'947f92f8-d82c-11e6-ab35-9801a7a75913'